In [22]:
#import pandas as pd
#import keras

In [38]:
#open file and make a  list of lines by splitting the text file at every occurance of '\n'.

f=  open("deu-eng.txt", 'r', encoding='utf-8') 
lines = f.read().split('\n')

In [40]:
#length 
len(lines)

169814

In [41]:
#need to split each line into input text and target text
input_texts = []
target_texts = []
#translation is being done by character to character 
input_characters = set()
target_characters = set()

In [42]:
num_samples = 10000
num_samples

10000

In [43]:
#loop over every sample we choose for training and fill the lists and sets. We'll also add '\t'
#(start-of-sequence) and '\n' (end-of-sequence) characters to every target text.
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [44]:
input_texts[155]

"It's me!"

In [45]:
target_texts[155]

"\tIch bin's.\n"

In [46]:
#characteristics of input and target sequences
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 87
Max sequence length for inputs: 16
Max sequence length for outputs: 53


In [47]:
print(input_characters)

[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [48]:
print(target_characters)

['\t', '\n', ' ', '!', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'Ä', 'Ö', 'Ü', 'ß', 'ä', 'é', 'ö', 'ü', 'ō', '’', '“', '„', '\u202f']


In [49]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [50]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [51]:
encoder_input_data.shape

(10000, 16, 70)

In [52]:
decoder_input_data.shape

(10000, 53, 87)

In [53]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [54]:
encoder_input_data[155].shape

(16, 70)

In [55]:
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [56]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [57]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [58]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [59]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [60]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 87)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 334848      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  352256      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [61]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('/results/seq2seq_eng-ger.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 58s 7ms/step - loss: 1.0235 - val_loss: 1.0630
Epoch 2/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.8009 - val_loss: 0.8607
Epoch 3/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.6765 - val_loss: 0.7659
Epoch 4/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.6130 - val_loss: 0.7099
Epoch 5/100
8000/8000 [==============================] - 49s 6ms/step - loss: 0.5673 - val_loss: 0.6807
Epoch 6/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.5324 - val_loss: 0.6431
Epoch 7/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.5037 - val_loss: 0.6221
Epoch 8/100
8000/8000 [==============================] - 52s 7ms/step - loss: 0.4803 - val_loss: 0.6031
Epoch 9/100
8000/8000 [==============================] - 48s 6ms/step - loss: 0.4604 - val_loss: 0.5939
Epoch 10/100
800

OSError: Unable to create file (unable to open file: name = '/results/seq2seq_eng-ger.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [65]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('C://Users//Ayushi Agarwal//Downloads//seq2seq_eng-ger.h5')

In [66]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

In [67]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

In [68]:
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
     # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence
      

In [69]:
for seq_index in range(10):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Fire!
Decoded sentence: Feuer!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Stop!
Decoded sentence: Stopp!

-
Input sentence: Wait!
Decoded sentence: Warte!

-
Input sentence: Go on.
Decoded sentence: Mach weiter.

-
Input sentence: Hello!
Decoded sentence: Hallo!



In [70]:
input_sentence = "How are you?"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
  test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

How are you?
Wie geht es dir?



In [71]:
val_input_texts = []
val_target_texts = []
line_ix = 12000
for line in lines[line_ix:line_ix+10]:
  input_text, target_text = line.split('\t')
  val_input_texts.append(input_text)
  val_target_texts.append(target_text)

val_encoder_input_data = np.zeros(
  (len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(val_input_texts):
  for t, char in enumerate(input_text):
    val_encoder_input_data[i, t, input_token_index[char]] = 1.

In [72]:
for seq_index in range(10):
  input_seq = val_encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', val_input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence[:-1])
  print('Ground Truth sentence:', val_target_texts[seq_index])

-
Input sentence: I bought the car.
Decoded sentence: Ich habe es versucht.
Ground Truth sentence: Ich habe das Auto gekauft.
-
Input sentence: I bowed politely.
Decoded sentence: Ich habe eine gefunden.
Ground Truth sentence: Ich verbeugte mich höflich.
-
Input sentence: I broke that one.
Decoded sentence: Ich habe es kaputtgemacht.
Ground Truth sentence: Ich habe das zerbrochen.
-
Input sentence: I came to say hi.
Decoded sentence: Ich kam wegen Tom.
Ground Truth sentence: Ich bin gekommen, um Hallo zu sagen.
-
Input sentence: I came yesterday.
Decoded sentence: Ich bin mit dem Bus gekommen.
Ground Truth sentence: Ich bin gestern gekommen.
-
Input sentence: I can bake bread.
Decoded sentence: Ich kann es schaffen.
Ground Truth sentence: Ich kann Brot backen.
-
Input sentence: I can be trusted.
Decoded sentence: Ich kann nicht schwimmen.
Ground Truth sentence: Man kann mir vertrauen.
-
Input sentence: I can carry that.
Decoded sentence: Ich kann es schaffen.
Ground Truth sentence: Ich